# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [3]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
# Display sample data
city_data_df.rename(columns = {'Unnamed: 0':"City_ID"}, inplace=True)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Mazyr,BY,52.0495,29.2456,49.66,88,100,8.90,overcast clouds
1,1,Chongwe,ZM,-15.3292,28.6820,76.59,22,2,6.33,clear sky
2,2,Cabinda,AO,-5.5500,12.2000,77.14,88,75,6.91,broken clouds
3,3,Saint-Philippe,RE,-21.3585,55.7679,71.19,78,26,10.65,scattered clouds
4,4,Ribeira Grande,PT,38.5167,-28.7000,70.20,83,75,6.91,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [4]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 75
What is the maximum temperature you would like for your vacation? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &\
                                       (city_data_df['Max Temp'] >= min_temp)]
                                    

# Display sample data
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Chongwe,ZM,-15.3292,28.6820,76.59,22,2,6.33,clear sky
2,2,Cabinda,AO,-5.5500,12.2000,77.14,88,75,6.91,broken clouds
7,7,Opuwo,NaN,-18.0607,13.8400,80.04,18,0,7.23,clear sky
8,8,West Bay,KY,19.3667,-81.4167,87.87,58,20,8.05,few clouds
13,13,Castro,BR,-24.7911,-50.0119,77.00,54,11,4.63,few clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Chongwe,ZM,-15.3292,28.6820,76.59,22,2,6.33,clear sky
2,2,Cabinda,AO,-5.5500,12.2000,77.14,88,75,6.91,broken clouds
8,8,West Bay,KY,19.3667,-81.4167,87.87,58,20,8.05,few clouds
13,13,Castro,BR,-24.7911,-50.0119,77.00,54,11,4.63,few clouds
24,24,Saint George,US,37.1041,-113.5841,79.79,26,0,1.99,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[['City', 'Country', 'Max Temp', 'Description', 'Lat', 'Lng']].copy()

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng
1,Chongwe,ZM,76.59,clear sky,-15.3292,28.6820
2,Cabinda,AO,77.14,broken clouds,-5.5500,12.2000
8,West Bay,KY,87.87,few clouds,19.3667,-81.4167
13,Castro,BR,77.00,few clouds,-24.7911,-50.0119
24,Saint George,US,79.79,clear sky,37.1041,-113.5841


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Chongwe,ZM,76.59,clear sky,-15.3292,28.6820,
2,Cabinda,AO,77.14,broken clouds,-5.5500,12.2000,
8,West Bay,KY,87.87,few clouds,19.3667,-81.4167,
13,Castro,BR,77.00,few clouds,-24.7911,-50.0119,
24,Saint George,US,79.79,clear sky,37.1041,-113.5841,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}      

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [14]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Chongwe,ZM,76.59,clear sky,-15.3292,28.6820,Grakas Lodge
2,Cabinda,AO,77.14,broken clouds,-5.5500,12.2000,Hotel Maiombe
8,West Bay,KY,87.87,few clouds,19.3667,-81.4167,Calypso Cove
13,Castro,BR,77.00,few clouds,-24.7911,-50.0119,CHACARA BAILLY
24,Saint George,US,79.79,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [56]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Chongwe,ZM,76.59,clear sky,-15.3292,28.6820,Grakas Lodge
2,Cabinda,AO,77.14,broken clouds,-5.5500,12.2000,Hotel Maiombe
8,West Bay,KY,87.87,few clouds,19.3667,-81.4167,Calypso Cove
13,Castro,BR,77.00,few clouds,-24.7911,-50.0119,CHACARA BAILLY
24,Saint George,US,79.79,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [50]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [60]:
# Set the file name.
output_data_file = 'WeatherPy_vacation.csv'

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [63]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [64]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Grakas Lodge</dd>\n<dt>City</dt><dd>Chongwe</dd>\n<dt>Country</dt><dd>ZM</dd>\n<dt>Current Weather</dt><dd>clear sky and 76.59 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Maiombe</dd>\n<dt>City</dt><dd>Cabinda</dd>\n<dt>Country</dt><dd>AO</dd>\n<dt>Current Weather</dt><dd>broken clouds and 77.14 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Calypso Cove</dd>\n<dt>City</dt><dd>West Bay</dd>\n<dt>Country</dt><dd>KY</dd>\n<dt>Current Weather</dt><dd>few clouds and 87.87 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>CHACARA BAILLY</dd>\n<dt>City</dt><dd>Castro</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>few clouds and 77.0 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Best Western Plus Abbey Inn</dd>\n<dt>City</dt><dd>Saint George</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>clear sky and 79.79 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Bungalow 63</dd>\n<dt>City</dt><dd>Hambantot

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [65]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
1,-15.3292,28.6820
2,-5.5500,12.2000
8,19.3667,-81.4167
13,-24.7911,-50.0119
24,37.1041,-113.5841
27,6.1241,81.1185
30,7.8500,9.7833
32,-1.1950,-46.0200
34,6.4965,2.6036
36,-17.7125,-39.2481


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [67]:

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))